In [ ]:
from Bio import SeqIO
import os
import subprocess
import sys

file_name = sys.argv[1]
location = "/mnt/griffin/saubar"

# file_name = f'{family}_{gene}_nostop.fas'
gene_name = file_name.split(".")[0]

original_fasta_file = SeqIO.parse(f"{location}/2.Individual_gene_files_BUSCO/{file_name}", 'fasta')

initial_output = ''
for original_sequences in original_fasta_file:
    if original_sequences.seq.translate()[-1] == "*":
        initial_output += f">{original_sequences.id}\n{original_sequences.seq[:-3]}\n\n"
    else:
        initial_output += f">{original_sequences.id}\n{original_sequences.seq}\n\n"
    
list_of_gene_folders = os.listdir(f"{location}/3.Busco_processing_individual_genes/")
if gene_name not in list_of_gene_folders:
    os.mkdir(f"{location}/3.Busco_processing_individual_genes/{gene_name}")

with open(f"{location}/3.Busco_processing_individual_genes/{gene_name}/{gene_name}_nostop.fas", 'w') as out_file:
    out_file.write(initial_output)


subprocess.run(f'cd {location}/3.Busco_processing_individual_genes/{gene_name}\nprank -d={gene_name}_nostop.fas -o={gene_name}_nostop.fas.aligned.fas -codon -iterate=10', shell = True, stderr = subprocess.DEVNULL)
new_location = f"{location}/3.Busco_processing_individual_genes/{gene_name}"


output1 = ""
output2 = ''


untranslated = SeqIO.parse(f"{new_location}/{gene_name}_nostop.fas.aligned.fas.best.fas", 'fasta')

for records in untranslated:
    if records.seq.translate()[-1] == "*":
        output1 += f">{records.id}\n{records.seq.replace('-','')[:-3]}\n\n"
        output2 += f">{records.id}\n{records.seq.translate()[:-1]+'-'}\n\n"
    else:
        output1 += f">{records.id}\n{records.seq.replace('-','')}\n\n"
        output2 += f">{records.id}\n{records.seq.translate()}\n\n"
# print(output1)
#mkdir(f"{location}/1.trimal_outputs")
try:
    os.mkdir(f"{location}/1.trimal_outputs")
except:
   pass
with open(f"{location}/1.trimal_outputs/{gene_name}_nostop.fas.aligned.fas.ungapped.fas", 'w') as out_file:
    out_file.write(output1)

with open(f"{location}/1.trimal_outputs/{gene_name}_nostop.fas.aligned.fas.translated.fas", 'w') as out_file:
    out_file.write(output2)

subprocess.run(f'cd {location}/1.trimal_outputs\ntrimal -colnumbering -htmlout trimal_output.html -in {gene_name}_nostop.fas.aligned.fas.translated.fas -out {gene_name}_nostop.fas.aligned.fas.trimal.fas -automated1 > columns_kept.txt', shell = True)
subprocess.run(f'cd {location}/1.trimal_outputs\ntrimal -backtrans {gene_name}_nostop.fas.aligned.fas.ungapped.fas -htmlout trimal_output.html -in {gene_name}_nostop.fas.aligned.fas.translated.fas -out {gene_name}_nostop.fas.aligned.fas.trimal.fas -automated1', shell = True)

print(f'"prank" -d={file_name} -o={file_name}.aligned.fas -codon')
#assert False

try:
    os.mkdir(f"{new_location}/2.iqtree_outputs")
except:
   pass

subprocess.run(f'cd {location}/2.iqtree_outputs\ncp ../1.trimal_outputs/{gene_name}_nostop.fas.aligned.fas.trimal.fas ./', shell = True)
  
subprocess.run(f'cd {new_location}/2.iqtree_outputs\niqtree -s {gene_name}_nostop.fas.aligned.fas.trimal.fas -m MFP -t RANDOM -bb 1000 -alrt 1000 -bnni -nt AUTO', shell = True)

subprocess.run(f'cd {new_location}/2.iqtree_outputs\ncp ./{gene_name}_nostop.fas.aligned.fas.trimal.fas.treefile ../', shell = True)

subprocess.run(f'cd {new_location}\nHYPHYMPI absrel --alignment {gene_name}_nostop.fas.aligned.fas.trimal.fas --tree {gene_name}_nostop.fas.aligned.fas.trimal.fas.treefile ', shell = True)

# subprocess.run(f'cd {location}\nHYPHYMPI busted --alignment {file_name}.aligned.fas.trimal.fas --tree {file_name}.aligned.fas.trimal.fas.treefile ', shell = True, stderr = subprocess.DEVNULL)
# print(f"HYPHYMPI busted --alignment {file_name}.aligned.fas.best.fas --tree{file_name}.aligned.fas.best.fas.treefile ")

# subprocess.run(f'cd {location}\nHYPHYMPI busted --alignment {file_name}.aligned.fas.best.fas --tree {file_name}.aligned.fas.best.fas.treefile ', shell = True)

# subprocess.run(f'cd {location}\nHYPHYMPI fel --alignment {file_name}.aligned.fas.trimal.fas --tree {file_name}.aligned.fas.trimal.fas.treefile ', shell = True, stderr = subprocess.DEVNULL)
# #print(f"HYPHYMPI busted --alignment {file_name}.aligned.fas.best.fas --tree{file_name}.aligned.fas.best.fas.treefile ")

# subprocess.run(f'cd {location}\nHYPHYMPI fel --alignment {file_name}.aligned.fas.best.fas --tree {file_name}.aligned.fas.best.fas.treefile ', shell = True)

# subprocess.run(f'cd {location}\nHYPHYMPI meme --alignment {file_name}.aligned.fas.trimal.fas --tree {file_name}.aligned.fas.trimal.fas.treefile ', shell = True, stderr = subprocess.DEVNULL)
# #print(f"HYPHYMPI busted --alignment {file_name}.aligned.fas.best.fas --tree{file_name}.aligned.fas.best.fas.treefile ")

# subprocess.run(f'cd {location}\nHYPHYMPI meme --alignment {file_name}.aligned.fas.best.fas --tree {file_name}.aligned.fas.best.fas.treefile ', shell = True)

# subprocess.run(f'cd {location}\nHYPHYMPI absrel --alignment {file_name}.aligned.fas.trimal.fas --tree {file_name}.aligned.fas.trimal.fas.treefile ', shell = True, stderr = subprocess.DEVNULL)
# #print(f"HYPHYMPI busted --alignment {file_name}.aligned.fas.best.fas --tree{file_name}.aligned.fas.best.fas.treefile ")

# subprocess.run(f'cd {location}\nHYPHYMPI absrel --alignment {file_name}.aligned.fas.best.fas --tree {file_name}.aligned.fas.best.fas.treefile ', shell = True)
